In [126]:
import pandas as pd

df = pd.read_csv('train.csv')
df.head(10)


,Issue_id,Component,Title,Description,Status,Resolution,Priority
0,1700,Layout,font size 9px rounded down to 8px {font} {ll},There is a piece of code in the Windows GFX th...,VERIFIED,FIXED,0
1,7991,Plug-ins,Land OOP plugins in mozilla-central (preffed off),We need to land OOP plugins in mozilla-central...,RESOLVED,FIXED,0
2,1150,XPCOM,XP-COM Proxy code crashes when trying to AutoP...,(This bug imported from BugSplat; Netscapes in...,VERIFIED,FIXED,0
3,6143,XUL,crash setting display:none on grid rows [@ nsG...,User-Agent: Mozilla/5.0 (Windows; U; Win...,VERIFIED,FIXED,0
4,5952,Networking: Cache,limit the memory cache to a reasonable value,I have 1GB of RAM; so Im using 31MB of memory ...,VERIFIED,FIXED,0
5,6796,File Handling,protocol handling dialog,This bug is a first cut at the simplified cont...,VERIFIED,FIXED,0
6,1109,XUL,Tab widget showing all html elements from all ...,On a Mac; open messenger; select Address Book ...,VERIFIED,FIXED,0
7,110,Untriaged,broken links in mozillafirefox,Created attachment 648282; firefox.exe; ; User...,RESOLVED,INVALID,0
8,3195,Security: UI,Trunk; N610 crash [@ nsKeygenFormProcessor::Init],I was just browing through Jaimes talkback rep...,VERIFIED,WORKSFORME,0
9,7149,XPConnect,Browser only Regressions in ecma_3/Operators/1...,range <http://bonsai.mozilla.org/cvsquery.cgi?...,VERIFIED,FIXED,0


In [127]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52994 entries, 0 to 52993
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Issue_id     52994 non-null  int64 
 1   Component    52994 non-null  object
 2   Title        52994 non-null  object
 3   Description  52602 non-null  object
 4   Status       52994 non-null  object
 5   Resolution   52994 non-null  object
 6   Priority     52994 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 2.8+ MB
None


In [128]:
df['Priority'].value_counts()


Priority
2    31154
1     9412
0     8943
3     2287
4     1198
Name: count, dtype: int64

In [129]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52994 entries, 0 to 52993
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Issue_id     52994 non-null  int64 
 1   Component    52994 non-null  object
 2   Title        52994 non-null  object
 3   Description  52602 non-null  object
 4   Status       52994 non-null  object
 5   Resolution   52994 non-null  object
 6   Priority     52994 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 2.8+ MB


In [130]:
df['Title'] = df['Title'].astype('string')
df['Description'] = df['Description'].astype('string')

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52994 entries, 0 to 52993
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Issue_id     52994 non-null  int64 
 1   Component    52994 non-null  object
 2   Title        52994 non-null  string
 3   Description  52602 non-null  string
 4   Status       52994 non-null  object
 5   Resolution   52994 non-null  object
 6   Priority     52994 non-null  int64 
dtypes: int64(2), object(3), string(2)
memory usage: 2.8+ MB


In [131]:
df['Priority'] = df['Priority'].replace({4: 3})

print(df['Priority'].value_counts())


Priority
2    31154
1     9412
0     8943
3     3485
Name: count, dtype: int64


In [132]:
print(df.isnull().sum())


Issue_id         0
Component        0
Title            0
Description    392
Status           0
Resolution       0
Priority         0
dtype: int64


In [133]:
df = df.dropna(subset=['Description'])

print(df.isnull().sum())
print(f"New dataset size: {len(df)} rows")


Issue_id       0
Component      0
Title          0
Description    0
Status         0
Resolution     0
Priority       0
dtype: int64
New dataset size: 52602 rows


In [134]:
duplicate_count = df.duplicated(subset='Title').sum()
print(f"Number of duplicate titles: {duplicate_count}")


Number of duplicate titles: 527


In [135]:
duplicates = df[df.duplicated(subset='Title', keep=False)]
print(duplicates[['Title', 'Description', 'Priority']].head(10))


                                                  Title  \
73                                     Crash on startup   
98    crash in nsFontInflationData::FindFontInflatio...   
276       full page plugins dont work by file extension   
327                                               crash   
449                     broken links in mozilla firefox   
653     Crash in DOM timer code when visiting quios.com   
667                             Mac-Cannot backup certs   
936                     broken links in Mozilla Firefox   
997   Function in destructuring assignment cant see ...   
1171                         spellchecker doesnt launch   

                                            Description  Priority  
73    apprunner crashes on startup.  Stack trace:; ;...         0  
98    It first appeared in 14.0a1/20120419.; ; Signa...         0  
276   Full page plugins are not invoked for local fi...         0  
327   I load the page; and while the page is loading...         0  
449   Crea

### Machine Learning

In [136]:
df['Text'] = df['Title'] + ' ' + df['Description']


In [137]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', max_features=20000,ngram_range=(1, 2))

X = vectorizer.fit_transform(df['Text'])

y = df['Priority']


In [138]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
from sklearn.linear_model import LogisticRegression

custom_weights = {
    0: 2.0,   
    1: 1.5,   
    2: 1.0,   
    3: 1.0    
}

In [140]:
from sklearn.linear_model import LogisticRegression

# model = LogisticRegression(max_iter=1000, solver='lbfgs', multi_class='multinomial')
# model = LogisticRegression(max_iter=1000, solver='lbfgs', multi_class='multinomial', class_weight='balanced')
# model.fit(X_train, y_train)

model = LogisticRegression(
    max_iter=2000,
    solver='lbfgs',
    multi_class='multinomial',
    class_weight=custom_weights
)

model.fit(X_train, y_train)


c:\Users\DELL\Desktop\IndividualProject\hr200\machine-learning\venvML\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(class_weight={0: 2.0, 1: 1.5, 2: 1.0, 3: 1.0}, max_iter=2000,
                   multi_class='multinomial')

In [141]:
# y_pred = model.predict(X_test)

y_pred = model.predict(X_test)


In [142]:
# from sklearn.metrics import classification_report

# print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

from sklearn.metrics import classification_report
target_names = ["Low", "Medium", "High", "Critical"]
print(classification_report(y_test, y_pred, target_names=target_names))

# print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


              precision    recall  f1-score   support

         Low       0.43      0.47      0.45      1772
      Medium       0.39      0.24      0.30      1872
        High       0.72      0.86      0.78      6183
    Critical       0.47      0.05      0.10       694

    accuracy                           0.63     10521
   macro avg       0.50      0.41      0.41     10521
weighted avg       0.60      0.63      0.60     10521



In [143]:
df_cleaned = df[['Title', 'Description', 'Text', 'Priority']]
df_cleaned.to_csv('cleaned_bug_reports.csv', index=False)

print("Cleaned dataset saved successfully!")


Cleaned dataset saved successfully!
